In [1]:
from transformers import GPT2TokenizerFast
from copy import deepcopy
import numpy as np
from random import random

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

with open("asd.txt", "r") as f:
    all_txt = f.read()
def get_imput(txt, length=512):
    bi = int(random()*(len(txt)-1000))
    return txt[bi:bi+length]

/home/maciej/anaconda3/envs/tokenizex/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/maciej/anaconda3/envs/tokenizex/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
from copy import deepcopy

#dev not working, need positional encoding at the first step to take in to an account!

txt = "wikipediawolna"

def text_emb_pos(txt, tokenizer):
    emb_pos = []

    for i in range(1, len(txt)+1):
        et = tokenizer.encode(txt[:i])
        emb_pos.append(list(zip(et, list(range(len(et))))))

    return emb_pos

def create_mask(emb_pos):
    total_len = len(emb_pos)
    res = []
    mask = np.zeros([total_len, total_len])
    for embt, m in zip(emb_pos, mask):
        res.append(embt[-1])
        m[len(res)-1] = 1
        for ie in embt[:-1]:
            for j, je in enumerate(res[:-1]):
                if je == ie:
                    m[j] = 1
                    break
    return np.array(res), mask

def create_mask_embedding(txt, tokenizer):
    emb_pos = text_emb_pos(txt, tokenizer)
    res, mask = create_mask(emb_pos)    

    # return [e[0] for e in res], [e[1] for e in res], mask
    return res[:, 0], res[:, 1], mask

emb, pos, mask = create_mask_embedding(txt, tokenizer)


In [3]:
unicode_string = u"Hi!"
type(unicode_string)

str

In [4]:
original_string = "ż"
utf8_string = original_string.encode('utf-8')
 
print("Original String:", original_string)
print("UTF-8 String:", utf8_string)

Original String: ż
UTF-8 String: b'\xc5\xbc'


In [5]:
vocab = tokenizer.get_vocab()
i_vocab = {v: k for k, v in vocab.items()}
tokens = vocab.values()

In [6]:
def add_masks(o_mask_size, masks):
    iy = 0
    ix = 0
    o_mask = np.zeros((o_mask_size, o_mask_size))

    for m in masks:
        ms = len(m)
        o_mask[iy:iy+ms, ix:ix+ms] += m
        for i in range(iy+ms, o_mask_size):
            o_mask[i, ix:ix+ms] += m[-1]
        iy+=ms
        ix+=ms
    return o_mask

In [38]:
from time import time


def embedding_mask(txt, tokenizer):
    tokens = tokenizer.encode(txt)
    partial_masks = []

    res_emb = []
    res_pos = []
    pos_len = 0

    for t in tokens:
        td = tokenizer.decode(t)
        if len(td)==1 or "�" in td:
            emb = [t] 
            pos = np.zeros((1))
            mask = np.ones((1,1))
        else:
            emb, pos, mask = create_mask_embedding(td, tokenizer)
            if len(emb) == 1:
                raise Exception("Used!")
        res_emb.extend(emb)
        res_pos.extend(pos+pos_len)
        pos_len += pos[-1] + 1
        partial_masks.append(mask)

    return np.array(res_emb), res_pos, add_masks(len(res_emb), partial_masks)

txti = "hey ..."
# txti = "wikipediawolna"
# txti = "żółta żółć ..."

txti = tokenizer.decode(tokenizer.encode(txti))


C = 100
st = 0
for i in range(C):
    txti = tokenizer.decode(tokenizer.encode(get_imput(all_txt, 1024)))
    t1 = time()
    emb, pos, mask = embedding_mask(txti, tokenizer)
    st += time()-t1
    tok = tokenizer.encode(txti)
    lte = emb[mask[-1].astype(bool)]
    assert lte.tolist() == tok

my = st/C
print(my)




# lte.tolist() == tok, tok, lte

0.10814678430557251


In [42]:
C = 100
st = 0

txtil = []
for i in range(C):
    txt = get_imput(all_txt, 1024)
    t1 = time()
    txtil = tokenizer.encode(txt)
    st += time()-t1
    
default = st/C
print(default)

0.0002609133720397949


In [43]:
llm_examples = 128*500000

In [44]:
my/default

414.4930689450359

In [45]:
default

0.0002609133720397949

In [37]:
my

0.030662436485290528